
Car Price Prediction::
Download dataset from this link:

https://www.kaggle.com/hellbuoy/car-price-prediction

Problem Statement::
A Chinese automobile company Geely Auto aspires to enter the US market by setting up their manufacturing unit there and producing cars locally to give competition to their US and European counterparts.

They have contracted an automobile consulting company to understand the factors on which the pricing of cars depends. Specifically, they want to understand the factors affecting the pricing of cars in the American market, since those may be very different from the Chinese market. The company wants to know:

Which variables are significant in predicting the price of a car How well those variables describe the price of a car Based on various market surveys, the consulting firm has gathered a large data set of different types of cars across the America market.

task::
We are required to model the price of cars with the available independent variables. It will be used by the management to understand how exactly the prices vary with the independent variables. They can accordingly manipulate the design of the cars, the business strategy etc. to meet certain price levels. Further, the model will be a good way for management to understand the pricing dynamics of a new market.


In [114]:
import numpy as np
import pandas as pd
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras import layers


### 1.Load Data

In [115]:
data = pd.read_csv("CarPrice_Assignment.csv")

In [116]:
data.head()

,car_ID,symboling,CarName,fueltype,aspiration,doornumber,carbody,drivewheel,enginelocation,wheelbase,carlength,carwidth,carheight,curbweight,enginetype,cylindernumber,enginesize,fuelsystem,boreratio,stroke,compressionratio,horsepower,peakrpm,citympg,highwaympg,price
0,1,3,alfa-romero giulia,gas,std,two,convertible,rwd,front,88.6,168.8,64.1,48.8,2548,dohc,four,130,mpfi,3.47,2.68,9.0,111,5000,21,27,13495.0
1,2,3,alfa-romero stelvio,gas,std,two,convertible,rwd,front,88.6,168.8,64.1,48.8,2548,dohc,four,130,mpfi,3.47,2.68,9.0,111,5000,21,27,16500.0
2,3,1,alfa-romero Quadrifoglio,gas,std,two,hatchback,rwd,front,94.5,171.2,65.5,52.4,2823,ohcv,six,152,mpfi,2.68,3.47,9.0,154,5000,19,26,16500.0
3,4,2,audi 100 ls,gas,std,four,sedan,fwd,front,99.8,176.6,66.2,54.3,2337,ohc,four,109,mpfi,3.19,3.40,10.0,102,5500,24,30,13950.0
4,5,2,audi 100ls,gas,std,four,sedan,4wd,front,99.4,176.6,66.4,54.3,2824,ohc,five,136,mpfi,3.19,3.40,8.0,115,5500,18,22,17450.0


### 2.Check Missing Values ( If Exist ; Fill each record with mean of its feature )

In [117]:
data.isnull().sum()

car_ID              0
symboling           0
CarName             0
fueltype            0
aspiration          0
doornumber          0
carbody             0
drivewheel          0
enginelocation      0
wheelbase           0
carlength           0
carwidth            0
carheight           0
curbweight          0
enginetype          0
cylindernumber      0
enginesize          0
fuelsystem          0
boreratio           0
stroke              0
compressionratio    0
horsepower          0
peakrpm             0
citympg             0
highwaympg          0
price               0
dtype: int64

In [118]:
data.drop(["car_ID","CarName"],axis = 1 , inplace = True)

In [119]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 205 entries, 0 to 204
Data columns (total 24 columns):
 #   Column            Non-Null Count  Dtype  
---  ------            --------------  -----  
 0   symboling         205 non-null    int64  
 1   fueltype          205 non-null    object 
 2   aspiration        205 non-null    object 
 3   doornumber        205 non-null    object 
 4   carbody           205 non-null    object 
 5   drivewheel        205 non-null    object 
 6   enginelocation    205 non-null    object 
 7   wheelbase         205 non-null    float64
 8   carlength         205 non-null    float64
 9   carwidth          205 non-null    float64
 10  carheight         205 non-null    float64
 11  curbweight        205 non-null    int64  
 12  enginetype        205 non-null    object 
 13  cylindernumber    205 non-null    object 
 14  enginesize        205 non-null    int64  
 15  fuelsystem        205 non-null    object 
 16  boreratio         205 non-null    float64
 1

In [120]:
data = pd.get_dummies(data, columns = ["fueltype","aspiration","doornumber","carbody","drivewheel","enginelocation","enginetype","cylindernumber","fuelsystem"])

In [121]:
data.head()

,symboling,wheelbase,carlength,carwidth,carheight,curbweight,enginesize,boreratio,stroke,compressionratio,horsepower,peakrpm,citympg,highwaympg,price,fueltype_diesel,fueltype_gas,aspiration_std,aspiration_turbo,doornumber_four,doornumber_two,carbody_convertible,carbody_hardtop,carbody_hatchback,carbody_sedan,carbody_wagon,drivewheel_4wd,drivewheel_fwd,drivewheel_rwd,enginelocation_front,enginelocation_rear,enginetype_dohc,enginetype_dohcv,enginetype_l,enginetype_ohc,enginetype_ohcf,enginetype_ohcv,enginetype_rotor,cylindernumber_eight,cylindernumber_five,cylindernumber_four,cylindernumber_six,cylindernumber_three,cylindernumber_twelve,cylindernumber_two,fuelsystem_1bbl,fuelsystem_2bbl,fuelsystem_4bbl,fuelsystem_idi,fuelsystem_mfi,fuelsystem_mpfi,fuelsystem_spdi,fuelsystem_spfi
0,3,88.6,168.8,64.1,48.8,2548,130,3.47,2.68,9.0,111,5000,21,27,13495.0,0,1,1,0,0,1,1,0,0,0,0,0,0,1,1,0,1,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,1,0,0
1,3,88.6,168.8,64.1,48.8,2548,130,3.47,2.68,9.0,111,5000,21,27,16500.0,0,1,1,0,0,1,1,0,0,0,0,0,0,1,1,0,1,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,1,0,0
2,1,94.5,171.2,65.5,52.4,2823,152,2.68,3.47,9.0,154,5000,19,26,16500.0,0,1,1,0,0,1,0,0,1,0,0,0,0,1,1,0,0,0,0,0,0,1,0,0,0,0,1,0,0,0,0,0,0,0,0,1,0,0
3,2,99.8,176.6,66.2,54.3,2337,109,3.19,3.40,10.0,102,5500,24,30,13950.0,0,1,1,0,1,0,0,0,0,1,0,0,1,0,1,0,0,0,0,1,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,1,0,0
4,2,99.4,176.6,66.4,54.3,2824,136,3.19,3.40,8.0,115,5500,18,22,17450.0,0,1,1,0,1,0,0,0,0,1,0,1,0,0,1,0,0,0,0,1,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,1,0,0


### 3.Split into 50% Training(Samples,Labels) , 30% Test(Samples,Labels) and 20% Validation Data(Samples,Labels).

In [122]:
Input = data.drop("price",axis = 1)
Output = data[["price"]]

In [123]:
X_train = Input.iloc[:102,:]
Y_train = Output.iloc[:102,:]
X_test = Input.iloc[102:164,:]
Y_test = Output.iloc[102:164,:]
X_val = Input.iloc[164:,:]
Y_val = Output.iloc[164:,:]

### Normalization 

In [124]:
def normalize_X(x):
  mean = x.iloc[:,:14].mean(axis = 0)
  std = x.iloc[:,:14].mean(axis = 0)
  x.iloc[:,:14]-=mean
  x.iloc[:,:14]/=std

normalize_X(X_train)
normalize_X(X_test)
normalize_X(X_val)


/usr/local/lib/python3.7/dist-packages/pandas/core/indexing.py:1715: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  isetter(loc, v)


In [128]:
def normalize_Y(y):
  mean_Y = y.mean(axis = 0)
  std_Y = y.std(axis = 0)
  y-=mean_Y
  y/=std_Y
normalize_Y(Y_train)
normalize_Y(Y_test)
normalize_Y(Y_val)


### 4.Model : input Layer (No. of features ), 3 hidden layers including 10,8,6 unit & Output Layer with activation function relu/tanh (check by experiment)

In [129]:
model = Sequential()
model.add(layers.Dense(10,activation = "relu",input_shape = (X_train.shape[1],)))
model.add(layers.Dense(8,activation = "relu"))
model.add(layers.Dense(6,activation = "relu"))
model.add(layers.Dense(1))

### Compilation Step (Note : Its a Regression problem , select loss , metrics according to it) 6.Train the Model with Epochs (100) and validate it

In [130]:
model.compile(optimizer = "rmsprop",loss = "mse",metrics = ["mae"])

In [131]:
model.fit(X_train,Y_train,epochs = 20, validation_data = (X_val,Y_val),batch_size=20)

Epoch 1/20
6/6 [==============================] - 1s 52ms/step - loss: 0.8421 - mae: 0.6786 - val_loss: 0.8275 - val_mae: 0.7482
Epoch 2/20
6/6 [==============================] - 0s 9ms/step - loss: 0.8116 - mae: 0.6819 - val_loss: 0.8045 - val_mae: 0.7345
Epoch 3/20
6/6 [==============================] - 0s 9ms/step - loss: 0.9067 - mae: 0.6752 - val_loss: 0.7809 - val_mae: 0.7204
Epoch 4/20
6/6 [==============================] - 0s 9ms/step - loss: 0.7253 - mae: 0.6009 - val_loss: 0.7591 - val_mae: 0.7059
Epoch 5/20
6/6 [==============================] - 0s 9ms/step - loss: 0.8355 - mae: 0.6375 - val_loss: 0.7380 - val_mae: 0.6917
Epoch 6/20
6/6 [==============================] - 0s 9ms/step - loss: 0.9651 - mae: 0.6415 - val_loss: 0.7160 - val_mae: 0.6730
Epoch 7/20
6/6 [==============================] - 0s 10ms/step - loss: 0.7877 - mae: 0.5667 - val_loss: 0.6954 - val_mae: 0.6600
Epoch 8/20
6/6 [==============================] - 0s 9ms/step - loss: 0.9372 - mae: 0.6017 - val_loss:

### Evaluation Step

In [132]:
model.evaluate(X_test,Y_test)

2/2 [==============================] - 0s 4ms/step - loss: 0.6801 - mae: 0.4761


[0.6801491379737854, 0.47610652446746826]

### Prediction

In [133]:
model.predict(X_test)

array([[ 0.4778603 ],
       [ 0.46603546],
       [-0.02893355],
       [ 0.34936938],
       [ 0.17845218],
       [ 0.43301952],
       [ 0.48555353],
       [ 0.42112905],
       [ 0.52794415],
       [ 0.48210645],
       [ 0.4871187 ],
       [ 0.47406808],
       [ 0.5295093 ],
       [ 0.43360737],
       [ 0.4871187 ],
       [ 0.62198734],
       [-0.8234029 ],
       [-0.45296472],
       [-1.0789605 ],
       [-0.7754086 ],
       [-0.77077925],
       [-0.5162249 ],
       [-0.21978927],
       [-0.3650479 ],
       [ 0.35691014],
       [ 0.35691014],
       [ 0.1030917 ],
       [-0.06326113],
       [ 0.1363999 ],
       [-0.50839907],
       [-0.4776017 ],
       [ 0.10383067],
       [-0.46257198],
       [ 0.10706683],
       [ 0.06434304],
       [ 0.4498417 ],
       [-0.84190154],
       [-0.8250498 ],
       [-0.6076413 ],
       [-0.79644156],
       [-0.74580294],
       [ 0.04423963],
       [-0.18651548],
       [ 0.5584385 ],
       [-0.7915767 ],
       [-0